In [1]:
%matplotlib inline
import numpy as np
import _pickle as cPickle
import os
import gzip
import matplotlib.pyplot as plt

# First exercise: Classifying MNIST with MLPs
In this exercise you will implement a Neural Network (or MLP) and classify the MNIST digits with it.
MNIST is a "well hung" dataset that has been used a lot over the years to benchmark different classification algorithms. 
To learn more about it have a look here: http://yann.lecun.com/exdb/mnist/ .

# Data Loading
We first define a function for downloading and loading MNIST.
**WARNING**: Executing it will obviously use up some space on your machine ;). 

In [2]:
def mnist(datasets_dir='./data'):
    if not os.path.exists(datasets_dir):
        os.mkdir(datasets_dir)
    data_file = os.path.join(datasets_dir, 'mnist.pkl.gz')
    if not os.path.exists(data_file):
        print('... downloading MNIST from the web')
        try:
            import urllib
            urllib.urlretrieve('http://google.com')
        except AttributeError:
            import urllib.request as urllib
        url = 'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
        urllib.urlretrieve(url, data_file)

    print('... loading data')
    # Load the dataset
    f = gzip.open(data_file, 'rb')
    try:
        train_set, valid_set, test_set = cPickle.load(f, encoding="latin1")
    except TypeError:
        train_set, valid_set, test_set = cPickle.load(f)
    f.close()

    test_x, test_y = test_set
    test_x = test_x.astype('float32')
    test_x = test_x.astype('float32').reshape(test_x.shape[0], 1, 28, 28)
    test_y = test_y.astype('int32')
    valid_x, valid_y = valid_set
    valid_x = valid_x.astype('float32')
    valid_x = valid_x.astype('float32').reshape(valid_x.shape[0], 1, 28, 28)
    valid_y = valid_y.astype('int32')
    train_x, train_y = train_set
    train_x = train_x.astype('float32').reshape(train_x.shape[0], 1, 28, 28)
    train_y = train_y.astype('int32')
    rval = [(train_x, train_y), (valid_x, valid_y), (test_x, test_y)]
    print('... done loading data')
    return rval

# Neural Network Layers
We now define "bare bone" neural network layers.
The parts marked with **TODO** are where you should finish the implementation!
Conceptually we will implement the layers as follows:

Each layer has a constructor that takes an input layer plus some additional arguments such as layer size and the activation function name. The layer then uses the provided input layer to compute the layer dimensions, weight shapes, etc. and setup all auxilliary variables.

Each layer then has to provide three functions (as defined in the Layer class below): *output_shape()*, *fprop()* and *brop()*. The output_shape function is used to figure out the shape for the next layer and the *fprop()/bprop()* functions are used to compute forward and backward passes through the network.

In [3]:
# start by defining simple helpers
def sigmoid(x):
    return 1.0/(1.0+np.exp(-x))

def sigmoid_d(x):
    # TODO ################
    # TODO: implement this!
    # TODO ################
    inter=sigmoid(x)
    return (inter*(1.0-inter))
    #return x

def tanh(x):
    return np.tanh(x)

def tanh_d(x):
    # TODO ################
    # TODO: implement this!
    # TODO ################
    inter=np.tanh(x)
    return( 1.0 - np.square(inter))


def relu(x):
    #print("input--main",x)
    #dx=np.zeros_like(x)
    #dx[x > 0] = x
    return np.maximum(x,0.0)

def relu_d(x):
    # TODO ################
    # TODO: implement this!
    # TODO ################
    dx=np.zeros_like(x)
    dx[x >= 0] = 1. 
    return dx

def softmax(x, axis=1):
    # to make the softmax a "safe" operation we will 
    # first subtract the maximum along the specified axis
    # so that np.exp(x) does not blow up!
    # Note that this does not change the output.
    x_max = np.max(x, axis=axis, keepdims=True)
    x_safe = x - x_max
    e_x = np.exp(x_safe)
    return e_x / np.sum(e_x, axis=axis, keepdims=True)

def one_hot(labels):
    """this creates a one hot encoding from a flat vector:
    i.e. given y = [0,2,1]
     it creates y_one_hot = [[1,0,0], [0,0,1], [0,1,0]]
    """
    classes = np.unique(labels)
    n_classes = classes.size
    one_hot_labels = np.zeros(labels.shape + (n_classes,))
    for c in classes:
        one_hot_labels[labels == c, c] = 1
    return one_hot_labels

def unhot(one_hot_labels):
    """ Invert a one hot encoding, creating a flat vector """
    return np.argmax(one_hot_labels, axis=-1)

# then define an activation function class
class Activation(object):
    
    def __init__(self, tname):
        if tname == 'sigmoid':
            self.act = sigmoid
            self.act_d = sigmoid_d
        elif tname == 'tanh':
            self.act = tanh
            self.act_d = tanh_d
        elif tname == 'relu':
            self.act = relu
            self.act_d = relu_d
        else:
            raise ValueError('Invalid activation function.')
            
    def fprop(self, input):
        # we need to remember the last input
        # so that we can calculate the derivative with respect
        # to it later on
        self.last_input = input #z0
        return self.act(input) #h0=g0(z0)
    
    def bprop(self, output_grad):
        #return np.dot (self.act_d(self.last_input).T, output_grad ) #dl/dz
        return  output_grad * self.act_d(self.last_input)  #dl/dz # hadamard product

# define a base class for layers
class Layer(object):
    
    def fprop(self, input):
        """ Calculate layer output for given input 
            (forward propagation). 
        """        
        raise NotImplementedError('This is an interface class, please use a derived instance')

    def bprop(self, output_grad):
        """ Calculate input gradient and gradient 
            with respect to weights and bias (backpropagation). 
        """
        raise NotImplementedError('This is an interface class, please use a derived instance')

    def output_size(self):
        """ Calculate size of this layer's output.
        input_shape[0] is the number of samples in the input.
        input_shape[1:] is the shape of the feature.
        """
        raise NotImplementedError('This is an interface class, please use a derived instance')

# define a base class for loss outputs
# an output layer can then simply be derived
# from both Layer and Loss 
class Loss(object):

    def loss(self, output, output_net):
        """ Calculate mean loss given real output and network output. """
        raise NotImplementedError('This is an interface class, please use a derived instance')

    def input_grad(self, output, output_net):
        """ Calculate input gradient real output and network output. """
        raise NotImplementedError('This is an interface class, please use a derived instance')

# define a base class for parameterized things        
class Parameterized(object):
    
    def params(self):
        """ Return parameters (by reference) """
        raise NotImplementedError('This is an interface class, please use a derived instance')
    
    def grad_params(self):
        """ Return accumulated gradient with respect to params. """
        raise NotImplementedError('This is an interface class, please use a derived instance')

# define a container for providing input to the network
class InputLayer(Layer):
    
    def __init__(self, input_shape):
        if not isinstance(input_shape, tuple):
            raise ValueError("InputLayer requires input_shape as a tuple")
        self.input_shape = input_shape

    def output_size(self):
        return self.input_shape
    
    def fprop(self, input):
        return input
    
    def bprop(self, output_grad):
        return output_grad
        
class FullyConnectedLayer(Layer, Parameterized):
    """ A standard fully connected hidden layer, as discussed in the lecture.
    """
    
    def __init__(self, input_layer, num_units, 
                 init_stddev, activation_fun=Activation('relu')):
        self.num_units = num_units
        self.activation_fun = activation_fun
        # the input shape will be of size (batch_size, num_units_prev) 
        # where num_units_prev is the number of units in the input 
        # (previous) layer
        self.input_shape = input_layer.output_size()
        
        #print("printing details of each layer\n")
        #print("input shape",self.input_shape)
        #print("number of units", self.num_units)
        
        
       # self.last_input = input
        # TODO ################################
        # TODO: implement weight initialization
        # TODO ################################
        # this is the weight matrix it should have shape: (num_units_prev, num_units)
        #print (self.input_shape[1])
        #print (self.num_units)
        self.W = np.random.randn(self.input_shape[1],self.num_units) * init_stddev #FIXME
        #self.W = self.W /float(self.W.shape[0])
        #print("shape of weight ",self.W.shape)
        # and this is the bias vector of shape: (num_units)

        self.b = np.random.randn(self.num_units)* init_stddev  #FIXME
        #self.b= self.b/float(self.b.shape[0])
        #print("shape of bias ",self.b.shape)
        # create dummy variables for parameter gradients
        # no need to change these here!
        self.dW = None
        self.db = None
    
    def output_size(self):
        return (self.input_shape[0], self.num_units)
    
    def fprop(self, input):
        # TODO ################################################
        # TODO: implement forward propagation
        # NOTE: you should also handle the case were 
        #       activation_fun is None (meaning no activation)
        #       then this is simply a linear layer
        # TODO ################################################
        # you again want to cache the lasfpropt_input for the bprop
        # implementation below!
        self.last_input = input
        
        ##print("shape of X",input.shape)
        #print("shape of weight",self.W.shape)
        #print("shape of bias",self.b.shape)
        
        
        #print("printing input in fprop---------", self.last_input)
        
        #print("shape of last input",self.last_input.shape)
        # NOTE: Use numpy dot product
        # h = t(a), a = W * x + b 
        activation=np.dot(self.last_input,self.W)+self.b #a = W * x + b : in matrix notation - a= XW+b
        if self.activation_fun is None :
            hidden_layer=activation  # h = t(a)
        else  :  
            #a=Activation(activation_fun)
            hidden_layer= self.activation_fun.fprop(activation)  # h = t(a)
        return  hidden_layer
        #FIXME
        raise NotImplementedError("You should implement this!")
        return np.zeros(self.output_size)
        
    def bprop(self, output_grad):
        """ Calculate input gradient (backpropagation). """
        #import pdb; pdb.set_trace()
        # TODO ################################
        # TODO: implement backward propagation
        # TODO ###############################
        
        # HINT: you may have to divide dW and db by n
        #       to make gradient checking work 
        #       OR you divide the gradient in the output layer by n
        n = output_grad.shape[0]
        # accumulate gradient wrt. the parameters first
        # we will need to store these to later update
        # the network after a few forward backward passes
        # NOTE: you should also handle the case were 
        #       activation_fun is None (meaning no activation)
        # the gradient wrt. W should be stored as self.dW
        # the gradient wrt. b should be stored as selfdb
       
        # NOTE: self.dW is also a numpy dot product
        
        #self.dW = np.dot(np.dot(output_grad,self.act_d(self.last_input)),self.fprop(input))/n #FIXME
        #self.db = np.dot(output_grad,self.act_d(self.last_input))/n #FIXME
        #a=Activation(activation_fun)
        
        #self.dW = np.dot(self.activation_fun.fprop(self.activation_fun.last_input).T,
        #                 self.activation_fun.bprop(output_grad)  )#h.T*dl/dz
        
        
        #print("last input in bprop------",self.last_input)
        
       # print("activation function in bprop--",self.activation_fun)
        
        if self.activation_fun is None:
            inter = output_grad
        else:
            inter = self.activation_fun.bprop(output_grad) # hadamard product, dl/dz
        
        grad_input = np.dot(inter,self.W.T)
        
        self.dW = np.dot(self.last_input.T,inter)
        
        #self.db = inter/n #dl/dz
        self.db = np.dot(np.ones((1, n)), inter)
        
        
        #print("dw:",self.dW)
        #print("db:",self.db)
        
        # the gradient wrt. the input should be calculated here
        
        #print("printing shapes for b prop")
        #print(output_grad.shape)
        #print(self.W.T.shape)
        #print(self.activation_fun.act_d(self.activation_fun.last_input).shape)
        
        #maybe we have to consider the weights and activation gradients of prev layer
        ##############check this!!!!!!!!!!!!!!!!!!!!!!!!!!!###########
        
        #grad_input=  np.dot(output_grad ,self.W.T ,
        #self.activation_fun.act_d(self.activation_fun.last_input))
        #grad_input = np.dot(self.activation_fun.act_d(self.activation_fun.last_input),self.W)
        
        #activation=np.dot(self.last_input,self.W)+self.b
        
        #grad_input=np.dot(self.activation_fun.act_d(activation),self.W.T)

        return grad_input
      
        raise NotImplementedError("you should implement this")
        
    def params(self):
        return self.W, self.b

    def grad_params(self):
        
        #print("inside grad_params")
        #print("dw:",self.dW)
        #print("db:",self.db)
        
        return self.dW, self.db

# finally we specify the interface for output layers 
# which are layers that also have a loss function
# we will implement two output layers:
#  a Linear, and Softmax (Logistic Regression) layer
# The difference between output layers and and normal 
# layers is that they will be called to compute the gradient
# of the loss through input_grad(). bprop will never 
# be called on them!
class LinearOutput(Layer, Loss):
    """ A simple linear output layer that  
        uses a squared loss (e.g. should be used for regression)
    """
    def __init__(self, input_layer):
        self.input_size = input_layer.output_size()
        
    def output_size(self):
        return (1,)
        
    def fprop(self, input):
        return input

    def bprop(self, output_grad):
        raise NotImplementedError(
            'LinearOutput should only be used as the last layer of a Network'
            + ' bprop() should thus never be called on it!'
        )
    
    def input_grad(self, Y, Y_pred):
        # TODO ####################################
        # TODO: implement gradient of squared loss
        # TODO ####################################
        #return np.zeros_like(Y_pred) #FIXME
        n_samples = Y.shape[0]
        return (Y_pred-Y)*1.0/n_samples

    def loss(self, Y, Y_pred):
        loss = 0.5 * np.square(Y_pred - Y)
        return np.mean(np.sum(loss, axis=1))

class SoftmaxOutput(Layer, Loss):
    """ A softmax output layer that calculates 
        the negative log likelihood as loss
        and should be used for classification.
    """
    
    def __init__(self, input_layer):
        self.input_size = input_layer.output_size()
        
    def output_size(self):
        return (1,)
    
    def fprop(self, input):
        return softmax(input)
    
    def bprop(self, output_grad):
        raise NotImplementedError(
            'SoftmaxOutput should only be used as the last layer of a Network'
            + ' bprop() should thus never be called on it!'
        )
    
    def input_grad(self, Y, Y_pred):
        # TODO #######################################################
        # TODO: implement gradient of the negative log likelihood loss
        # TODO #######################################################
        # HINT: since this would involve taking the log 
        #       of the softmax (which is np.exp(x)/np.sum(x, axis=1))
        #       this gradient computation can be simplified a lot,
        #       you may find a connection to the LinearOutput layer!
        # return np.zeros_like(Y_pred)
        #return(-np.dot(Y,1/(Y_pred)) + np.dot((1-Y),1/(1-Y_pred)))
        n_samples = Y.shape[0]
        return (Y_pred-Y)*1.0/n_samples
             

    def loss(self, Y, Y_pred):
        # Assume one-hot encoding of Y
        out = Y_pred
        # to make the loss numerically stable 
        # you should add an epsilon in the log ;)
        eps = 1e-10
        # TODO ####################################
        # calculate negative log likelihood
        # TODO ####################################
        #loss= (Y)* np.log(out+eps) # + np.dot((1-Y),np.log(1-Y_pred+eps))
        #loss = np.zeros_like(out) #FIXME
        
        
        #print("loss", -1*np.mean(np.sum(loss)))
        loss = -np.sum(np.multiply(Y, np.log(out + eps)), axis=1)
        #return -1*np.mean(np.sum(loss))
        return np.mean(loss)
        

# Neural Network class
With all layers in place (and properly implemented by you) we can finally define a neural network.
For our purposes a neural network is simply a collection of layers which we will cycle through and on which we will call fprop and bprop to compute partial derivatives with respect to the input and the parameters.

Pay special attention to the *check_gradients()* function in which you should implement automatic differentiation. This function will become your best friend when checking the correctness of your implementation.

In [4]:

class NeuralNetwork:
    """ Our Neural Network container class.
    """
    def __init__(self, layers):
        self.layers = layers
        
    def _loss(self, X, Y):
        Y_pred = self.predict(X)
        return self.layers[-1].loss(Y, Y_pred)

    def predict(self, X):
        """ Calculate an output Y for the given input X. """
        # TODO ##########################################
        # TODO: implement forward pass through all layers
        # TODO ##########################################
        input_here = X
        #loop through all layers in self.layers and calculate the complete chain
        n = len(self.layers)
        for i in range(1, n):
            input_here = self.layers[i].fprop(input_here)

        return input_here
    
    def backpropagate(self, Y, Y_pred, upto=0):
        """ Backpropagation of partial derivatives through 
            the complete network up to layer 'upto'
        """
        next_grad = self.layers[-1].input_grad(Y, Y_pred)
        # TODO ##########################################
        # TODO: implement backward pass through all layers
        # TODO ##########################################
        #i = np.shape(self.layers)[0]
        #print("shape of i is ",i)
        #i=i-1
        #while i>=upto:
            #print("input to gradient is ", self.layers[i-1].last_input)
         #   a=self.layers[i-1].activation_fun
          #  print(next_grad.shape)
           # print(self.layers[i-1].W.shape)
            #print(a.act_d(self.layers[i-1].last_input).shape)
            #next_grad=np.dot(next_grad,self.layers[i-1].W,a.act_d(self.layers[i-1].last_input))
            #i=i-1
            
            
        #iterate in reverse order for backpropagation and leave out the last entry
        n = len(self.layers)
        for i in range(n-2, upto-1, -1):
            next_grad = self.layers[i].bprop(next_grad)
        
        return next_grad
    
    def classification_error(self, X, Y):
        """ Calculate error on the given data 
            assuming they are classes that should be predicted. 
        """
        Y_pred = unhot(self.predict(X))
        error = Y_pred != unhot(Y)
        return np.mean(error)
    
    def sgd_epoch(self, X, Y, learning_rate, batch_size):
        n_samples = X.shape[0]
        n_batches = n_samples // batch_size
        
        for b in range(n_batches):
            # TODO #####################################
            # Implement stochastic gradient descent here
            # TODO #####################################
            # start by extracting a batch from X and Y
            # (you can assume the inputs are already shuffled)

            # TODO: then forward and backward propagation + updates
            # HINT: layer.params() returns parameters *by reference*
            #       so you can easily update in-place
            # Determine the sample slice for this minibatch
            batch_start = b * batch_size
            batch_end = (b + 1) * batch_size
        
            # Extract the batch from the training set
            X_batch = X[batch_start:batch_end]
            #print("X_batch size",X_batch.shape)
            Y_batch = Y[batch_start:batch_end]
            #print("Y_batch size",Y_batch.shape)
         
            Y_pred = self.predict(X_batch)
            self.backpropagate(Y_batch, Y_pred, upto=0)
            num_lay = len(self.layers)
            for i in range(1, num_lay - 1):
                w, b = self.layers[i].params()
                dw, db = self.layers[i].grad_params()
                w -= learning_rate * dw
                b -= learning_rate * np.transpose(np.squeeze(db))
                self.layers[i].W = w
                self.layers[i].b = b

                
                
    def adam(self, X, Y, learning_rate, batch_size,decay_rate1= 0.01,decay_rate2=0.02):
        n_samples = X.shape[0]
        n_batches = n_samples // batch_size
        moment1=0
        moment2=0
        
        for b in range(n_batches):
            # TODO #####################################
            # Implement stochastic gradient descent here
            # TODO #####################################
            # start by extracting a batch from X and Y
            # (you can assume the inputs are already shuffled)

            # TODO: then forward and backward propagation + updates
            # HINT: layer.params() returns parameters *by reference*
            #       so you can easily update in-place
            # Determine the sample slice for this minibatch
            batch_start = b * batch_size
            batch_end = (b + 1) * batch_size
        
            # Extract the batch from the training set
            X_batch = X[batch_start:batch_end]
            Y_batch = Y[batch_start:batch_end]
            Y_pred = self.predict(X_batch)
            self.backpropagate(Y_batch, Y_pred, upto=0)
            num_lay = len(self.layers)
            for i in range(1, num_lay - 1):
                w, b = self.layers[i].params()
                dw, db = self.layers[i].grad_params()
                
                moment1_= decay_rate1 * moment1 + (1-decay_rate1)*dw
                moment2_= decay_rate2 * moment2 + (1-decay_rate2)*np.multiply(dw,dw)
                delta=moment1_*1/(np.sqrt(0.001+moment2_))     
                w -= learning_rate*delta 
                
                moment1= decay_rate1 * moment1 + (1-decay_rate1)*db
                moment2= decay_rate2 * moment2 + (1-decay_rate2)*db*db
                delta=moment1*1/(np.sqrt(0.001+moment2))                
                
                print("shape of delta",delta.shape)
                print("shape of b",b.shape)  
                
                b -= learning_rate * np.squeeze(delta)
                self.layers[i].W = w
                self.layers[i].b = b
                   

                    
    def gd_epoch(self, X, Y,learning_rate):
        # TODO ##################################################
        # Implement batch gradient descent here
        # A few hints:
        #   There are two strategies you can follow:
        #   Either shove the whole dataset throught the network
        #   at once (which can be problematic for large datasets)
        #   or run through it batch wise as in the sgd approach
        #   and accumulate the gradients for all parameters as
        #   you go through the data. Either way you should then
        #   do one gradient step after you went through the
        #   complete dataset!
        # TODO ##################################################
        Y_pred = self.predict(X)
        self.backpropagate(Y, Y_pred, upto=0)
        num_lay = len(self.layers)
        for i in range(1, num_lay - 1):
                w, b = self.layers[i].params()
                dw, db = self.layers[i].grad_params()
                w -= learning_rate * dw
                b -= learning_rate * np.transpose(np.squeeze(db))
                self.layers[i].W = w
                self.layers[i].b = b
                   
       
    
    def train(self, X, Y, X_valid, Y_valid, X_test,Y_test ,learning_rate, max_epochs, batch_size,
              descent_type, y_one_hot=True):

        """ Train network on the given data. """
        n_samples = X.shape[0]
        n_batches = n_samples // batch_size
        if y_one_hot:
            Y_train = one_hot(Y)
            Y_valid = one_hot(Y_valid)
            Y_test = one_hot(Y_test)
        else:
            Y_train = Y
            
        training_loss =np.ones(max_epochs+1) * np.nan 
        validation_loss = np.ones(max_epochs+1) * np.nan 
        train_error =np.ones(max_epochs+1) * np.nan 
        valid_error = np.ones(max_epochs+1) * np.nan
       
        
        print("... starting training")
        for e in range(max_epochs+1):
            if descent_type == "sgd":
                self.sgd_epoch(X, Y_train, learning_rate, batch_size)
            elif descent_type == "gd":
                self.gd_epoch(X, Y_train, learning_rate)
            elif descent_type == "adam":
                self.adam(X, Y_train, learning_rate,batch_size)
            else:
                raise NotImplementedError("Unknown gradient descent type {}".format(descent_type))

            # Output error on the training data
            train_loss = self._loss(X, Y_train)
            training_loss[e]=train_loss
            train_error[e] = self.classification_error(X, Y_train)
            te = train_error[e]
            validation_loss[e]=self._loss(X_valid,  Y_valid)
            v=validation_loss[e]
            validation_error=self.classification_error(X_valid, Y_valid)
            valid_error[e]=validation_error
            
            if(e%5==0):
                print('epoch {:.1f}, train loss {:.4f}, train error {:.4f}, validation loss {:.4f}, validation error {:.4f}'.format(
                    e, train_loss, te,v,validation_error))
            # TODO ##################################################
            # compute error on validation data:
            # simply make the function take validation data as input
            # and then compute errors here and print them
            # TODO ##################################################
            #validation_loss=self._loss(X_valid, Y_valid)  
        print("learning rate: ", learning_rate)
        print("descent type", descent_type)
        print("validation error", valid_error[len(valid_error)-1])
        
        
        
        
        print("test error",  self.classification_error(X_test, Y_test))
 
        #plt.plot(training_loss, c='b', label='Training loss')
        #plt.plot(validation_loss, c='m', label='Validation loss')
        #plt.plot(train_error, c='g', label='Training error')
        #plt.plot(valid_error, c='r', label='Validation error')
        #plt.legend(loc='lower left')
        
        return training_loss,train_error,validation_loss,valid_error
        

        
    def check_gradients(self, X, Y):
        """ Helper function to test the parameter gradients for
        correctness. """
        for l, layer in enumerate(self.layers):
            if isinstance(layer, Parameterized):
                print('checking gradient for layer {}'.format(l))
                for p, param in enumerate(layer.params()):
                    # we iterate through all parameters
                    #print("parameters of layer--",param)
                    param_shape = param.shape
                    # define functions for conveniently swapping
                    # out parameters of this specific layer and 
                    # computing loss and gradient with these 
                    # changed parametrs
                    def output_given_params(param_new):
                        """ A function that will compute the output 
                            of the network given a set of parameters
                        """
                        # copy provided parameters
                        param[:] = np.reshape(param_new, param_shape)
                        # return computed loss
                        return self._loss(X, Y)

                    def grad_given_params(param_new):
                        """A function that will compute the gradient 
                           of the network given a set of parameters
                        """
                        # copy provided parameters
                        param[:] = np.reshape(param_new, param_shape)
                        # Forward propagation through the net
                        #pdb.set_trace()
                        Y_pred = self.predict(X)
                        # Backpropagation of partial derivatives
                        #pdb.set_trace()
                        self.backpropagate(Y, Y_pred, upto=l)
                        # return the computed gradient 
                        return np.ravel(self.layers[l].grad_params()[p])

                    # let the initial parameters be the ones that
                    # are currently placed in the network and flatten them
                    # to a vector for convenient comparisons, printing etc.
                    param_init = np.ravel(np.copy(param))
                    
                    # To debug you network's gradients use scipys
                    # gradient checking!
                    epsilon = 1e-5
                    import scipy.optimize
                    err = scipy.optimize.check_grad(output_given_params, 
                          grad_given_params,
                          param_init) 
                    print('diff scipy {:.2e}'.format(err))
                    assert(err < epsilon)
                    
                    # reset the parameters to their initial values
                    param[:] = np.reshape(param_init, param_shape)

# Gradient Checking
After implementing everything it is always a good idea to setup some layers and perform gradient
checking on random data. **Note** that this is only an example! It is not a useful network architecture ;). We also expect you to play around with this to test all your implemented components.

In [5]:
input_shape = (5, 10)
n_labels = 6
layers = [InputLayer(input_shape)]
#print(InputLayer(input_shape))

layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=15,
                init_stddev=0.1,
                activation_fun=Activation('relu')
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=6,
                init_stddev=0.1,
                activation_fun=Activation('tanh')
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=n_labels,
                init_stddev=0.1,
                activation_fun=Activation('relu')
))
layers.append(SoftmaxOutput(layers[-1]))
nn = NeuralNetwork(layers)

In [6]:
# create random data
X = np.random.normal(size=input_shape)
# and random labels
Y = np.zeros((input_shape[0], n_labels))
for i in range(Y.shape[0]):
    idx = np.random.randint(n_labels)
    Y[i, idx] = 1.

In [7]:
#import pdb; pdb.set_trace()

#nn.check_gradients(X, Y)

#nn.train(X,Y,X,Y,learning_rate=0.6, 
         #max_epochs=30, batch_size=64, y_one_hot=False)

# Training on MNIST
Finally we can let our network run on the MNIST dataset!

First load the data and reshape it.

In [8]:
# load
Dtrain, Dval, Dtest = mnist()
X_train, y_train = Dtrain
X_valid, y_valid = Dval
X_test, y_test = Dtest

... loading data
... done loading data


*Dtrain* contains 50k images which are of size 28 x 28 pixels. Hence:

In [9]:
print("X_train shape: {}".format(np.shape(X_train)))
print("y_train shape: {}".format(np.shape(y_train)))

X_train shape: (50000, 1, 28, 28)
y_train shape: (50000,)


y_train will automatically be converted in the *train()* function to one_hot encoding.


But we need to reshape X_train, as our Network expects flat vectors of size 28*28 as input!

In [10]:
X_train = X_train.reshape(X_train.shape[0], -1)
print("Reshaped X_train size: {}".format(X_train.shape))
X_valid = X_valid.reshape((X_valid.shape[0], -1))
print("Reshaped X_valid size: {}".format(X_valid.shape))
X_test = X_test.reshape((X_test.shape[0], -1))
print("Reshaped X_test size: {}".format(X_test.shape))

Reshaped X_train size: (50000, 784)
Reshaped X_valid size: (10000, 784)
Reshaped X_test size: (10000, 784)


Ah, much better ;-)! 

Now we can finally really start training a Network!


I pre-defined a small Network for you below. Again This is not really a good default and will not produce state of the art results. Please play around with this a bit. See how different activation functions and training procedures (gd / sgd) affect the result.

In [11]:
def plot(tl,vl,max_epochs):
    fig, ax = plt.subplots(1, 1)
    ax.set_title('Training progress')
    ax.set_xlim([0, max_epochs])
    ax.set_ylim([0, 1])
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Error/loss')

    if ax.lines:
        ax.lines[0].set_ydata(tl)
        ax.lines[1].set_ydata(vl)
        ax.set_ylim([0,  1])
    else:
        ax.plot(tl, c='b', label='Training loss')
        ax.plot(vl, c='m', label='Validation loss')
        ax.legend(loc='upper right')
    fig.canvas.draw()

In [ ]:
def draw_graphs(tl, te, vl, ve):    
    epochs = np.arange(tl.shape[0])
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12, 12))
    y = [tl, vl, te, ve]
    lab = ["Training Loss", "Validation Loss", "Training Error", "Validation Error"]
    #y=[tl,vl]
    #lab = ["Training Loss", "Validation Loss"]
    i = 0
    for row in ax:
        for col in row:
            col.plot(epochs, y[i])
            col.grid(True)
            col.set_xlabel("EPOCHS")
            col.set_ylabel(lab[i])
            # col.annotate(s=y[i], xy=zip(epochs, y[i]))
            i += 1

In [ ]:
import time

# Setup a small MLP / Neural Network
# we can set the first shape to None here to indicate that
# we will input a variable number inputs to the network
input_shape = (None, 28*28)
layers = [InputLayer(input_shape)]

layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=100,
                init_stddev=0.01,
                activation_fun=Activation('tanh')
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=500,
                init_stddev=0.01,
                activation_fun=Activation('relu')
))

layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=300,
                init_stddev=0.01,
                activation_fun=Activation('relu')
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=10,
                init_stddev=0.01,
                # last layer has no nonlinearity 
                # (softmax will be applied in the output layer)
                activation_fun=None
))
layers.append(SoftmaxOutput(layers[-1]))

nn = NeuralNetwork(layers)
# Train neural network
t0 = time.time()
tl, te, vl, ve=nn.train(X_train, y_train, X_valid, y_valid,X_test, y_test, learning_rate=0.15, 
         max_epochs=250, batch_size=512, y_one_hot=True,descent_type="sgd")
t1 = time.time()
print('Duration: {:.1f}s'.format(t1-t0))
print("2-Layer network with 500+300 HU")
draw_graphs(tl, te, vl, ve)
plot(tl,vl,max_epochs=250)

layers = [InputLayer(input_shape)]

layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=100,
                init_stddev=0.01,
                activation_fun=Activation('tanh')
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=500,
                init_stddev=0.01,
                activation_fun=Activation('relu')
))

layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=300,
                init_stddev=0.01,
                activation_fun=Activation('relu')
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=10,
                init_stddev=0.01,
                # last layer has no nonlinearity 
                # (softmax will be applied in the output layer)
                activation_fun=None
))
layers.append(SoftmaxOutput(layers[-1]))


#t0 = time.time()
#tl, te, vl, ve=nn.train(X_train, y_train, X_valid, y_valid,X_test, y_test, learning_rate=0.2, 
        # max_epochs=250, batch_size=512, y_one_hot=True,descent_type="gd")
#t1 = time.time()
#print('Duration: {:.1f}s'.format(t1-t0))
#print("2-Layer network with 500+200 HU")
#draw_graphs(tl, te, vl, ve)
#plot(tl,vl,max_epochs=250)

... starting training
epoch 0.0, train loss 2.3009, train error 0.8864, validation loss 2.3017, validation error 0.8936
epoch 5.0, train loss 1.5631, train error 0.5763, validation loss 1.5527, validation error 0.5703
epoch 10.0, train loss 0.3825, train error 0.1090, validation loss 0.3606, validation error 0.1052
epoch 15.0, train loss 0.2146, train error 0.0627, validation loss 0.2073, validation error 0.0607
epoch 20.0, train loss 0.1525, train error 0.0450, validation loss 0.1570, validation error 0.0461
epoch 25.0, train loss 0.1187, train error 0.0352, validation loss 0.1371, validation error 0.0405
epoch 30.0, train loss 0.0991, train error 0.0297, validation loss 0.1269, validation error 0.0396
epoch 35.0, train loss 0.0890, train error 0.0261, validation loss 0.1176, validation error 0.0359
epoch 40.0, train loss 0.0688, train error 0.0203, validation loss 0.1116, validation error 0.0324
epoch 45.0, train loss 0.0591, train error 0.0181, validation loss 0.1119, validation err

# Figure out a reasonable Network that achieves good performance
As the last part of this task, setup a network that works well and gets reasonable accuracy, say ~ 1-3 percent error on the **validation set**. 
Train this network on the complete data and compute the **test error**. 

Visualize the validation loss and training loss for each iteration in a plot, e.g. using matplotlib